In [1]:
import os
import matplotlib.pylab as plt
from matplotlib.patches import Rectangle
import numpy as np


from ipywidgets import Layout, interact, IntSlider, FloatSlider
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

import DL_Sequence

num_classes = 5
res = 32

dataset = DL_Sequence.iSCAT_DataGenerator(batch_size=4, epoch_size=1024, res=res, frames=32, thread_count=10,
                    PSF_path="../PSF_subpx_fl32.npy", exD=5000, devD=4000, exPT_cnt=500, devPT_cnt=499, exIntensity=1.0, devIntensity=0.3, target_frame=15,
                    num_classes=num_classes, verbose = 1)

data = dataset.samples
targets = dataset.particles_in_sight_cnt
poss = dataset.particle_positions
pt_cnts = dataset.pt_cnts
dataset.destroy()


ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [4]:
import tensorflow as tf

model_name = "models/model_dense_5000_simple"
model = tf.keras.models.load_model(model_name + ".h5")
predictions = tf.argmax(model.predict(data), axis = 1)
predictions = np.array(predictions)

print(data.shape, targets.shape)

print(np.max(data), np.min(data))
print(np.mean(targets), np.std(targets))

print("Accuracy: ", np.mean(predictions == targets))

32/32 [==============================] - 0s 9ms/step
(1024, 32, 32, 32) (1024,)
1.0 0.0
1.9521484375 1.4693001873226277
Accuracy:  0.419921875


In [3]:
sample_inds = np.insert(np.cumsum(pt_cnts), 0, 0)

sample_size_px = 480
FOV_edge = sample_size_px / 2 - res / 2

def plot_step_fast(step, sample_cur):    
    imgs = data[sample_cur]
#     plt.imshow(imgs[step, :, :])
#     plt.axis([-0.5, data.shape[2]-0.5, -0.5, data.shape[2]-0.5])
#     plt.title("data")
#     plt.show()
    
    begin = sample_inds[sample_cur]
    end = begin + pt_cnts[sample_cur]
    x = poss[step, begin:end, 1]
    y = poss[step, begin:end, 0]
    
    plt.subplot(1, 2, 1)
    plt.axis([-FOV_edge, sample_size_px-FOV_edge, -FOV_edge, sample_size_px-FOV_edge])
    plt.gca().add_patch(Rectangle((0, 0), res, res, edgecolor = 'red', fill=False))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.scatter(x,y)
    
    plt.subplot(1, 2, 2)
    plt.imshow(imgs[step, :, :])
    plt.axis([-0.5, res-0.5, -0.5, res-0.5])
    plt.scatter(x,y, c="red")
    plt.title("data")
    plt.show()
    
    print("gold:", targets[sample_cur])
    print("pred:", int(predictions[sample_cur]))


    
interact(plot_step_fast, step=IntSlider(min=0, max=data.shape[1] - 1), sample_cur=IntSlider(min=0, max=data.shape[0] - 1));

interactive(children=(IntSlider(value=0, description='step', max=31), IntSlider(value=0, description='sample_c…